In [1]:
method = "text-img"

In [2]:
import pandas as pd
import torch
import transformers
import torch
from tqdm import tqdm
import numpy as np
from pytorch_fid import fid_score
import os
import shutil


# ASR

In [ ]:
path = f"../output/{method}/output_prompt_10.csv"
df = pd.read_csv(path, index_col=0)
df = df.dropna()
all_count = df.shape[0]
all_success = df[df.nsfw].shape[0]
print(f'ASR: {all_success / all_count *100 :.2f}')

# CLIP similarity

In [ ]:
path = f"../output/{method}/output_prompt_10.csv"
df = pd.read_csv(path, index_col=0)
df = df.dropna()

clip_sim = df[df.bypass].clip_similarity.mean()
print(f'CLIP sim: {clip_sim :.4f}')

# PPL

In [ ]:
path = f"../output/{method}/output_prompt_10.csv"
model_id = "meta-llama/Meta-Llama-3.1-8B-Instruct"
model = transformers.AutoModelForCausalLM.from_pretrained(model_id, torch_dtype=torch.bfloat16)
tokenizer = transformers.AutoTokenizer.from_pretrained(model_id)
device = torch.device('cuda:5')
model = model.to(device)

def calculate_ppl(prompt):
    encodings = tokenizer(prompt, return_tensors="pt")

    max_length = 1024
    stride = 512
    seq_len = encodings.input_ids.size(1)
    nlls = []
    prev_end_loc = 0
    for begin_loc in range(0, seq_len, stride):
        end_loc = min(begin_loc + max_length, seq_len)
        trg_len = end_loc - prev_end_loc 
        input_ids = encodings.input_ids[:, begin_loc:end_loc].to(device)
        target_ids = input_ids.clone()
        target_ids[:, :-trg_len] = -100
        with torch.no_grad():
            outputs = model(input_ids, labels=target_ids)
            neg_log_likelihood = outputs.loss

        nlls.append(neg_log_likelihood)

        prev_end_loc = end_loc
        if end_loc == seq_len:
            break

    ppl = torch.exp(torch.stack(nlls).mean())
    return ppl

df = pd.read_csv(path, index_col=0)
df = df.dropna()
grouped = df.groupby('index')

ppl = []
for _, row in tqdm(df.iterrows()):
    ppl.append(calculate_ppl(row.prompt).detach().cpu().numpy())

print(f"PPL: {np.median(ppl)}")

# FID

In [ ]:
target_path = f"../output/{method}/output_fig/10"

ref_folder = os.path.join(target_path, 'ref_images')
generate_folder = os.path.join(target_path, 'generate_images')

os.makedirs(ref_folder, exist_ok=True)
os.makedirs(generate_folder, exist_ok=True)

for filename in os.listdir(target_path):
    file_path = os.path.join(target_path, filename)

    if os.path.isfile(file_path):
        if filename.endswith('_ref.png'):
            shutil.move(file_path, os.path.join(ref_folder, filename))
        else:
            shutil.move(file_path, os.path.join(generate_folder, filename))
fid_value = fid_score.calculate_fid_given_paths([ref_folder, generate_folder], batch_size=256, dims=2048, device='cuda')
print(f'FID: {fid_value}')